In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt

#make plots inline using jupyter magic
%matplotlib inline

import pandas as pd
from pandas.plotting import scatter_matrix
from sklearn import datasets, linear_model, metrics


import matplotlib as mpl
import seaborn as sns

import sklearn.linear_model as skl_lm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.neighbors import KNeighborsClassifier
#Balanced RF Classifier
from imblearn.ensemble import BalancedRandomForestClassifier as BRF

from IPython.display import Markdown as md  #enable markdown within code cell
from IPython.display import display, Math, Latex

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import confusion_matrix
import time

from sklearn.metrics import balanced_accuracy_score, precision_score, recall_score, precision_recall_curve, make_scorer,f1_score
from sklearn.metrics import precision_recall_curve as PRC

from scipy.spatial.distance import pdist
from scipy.spatial.distance import cosine
from scipy.spatial.distance import cdist

from collections import Counter

from scipy.stats import norm
import scipy
from io import StringIO

import metrics
import ijson

data_dir = '/run/media/mnewlin/_userdata/uhnds/'
original_netflow_data_dir = data_dir + 'network/extracted/'
original_netflow_file = 'netflow_day-02'
fake_dir = '/run/media/mnewlin/_userdata/uhnds/network/converted/fake/'
real_dir = '/run/media/mnewlin/_userdata/uhnds/network/converted/real/'
real_file = 'netflow_day-02'

pd.show_versions()


INSTALLED VERSIONS
------------------
commit: None
python: 3.7.1.final.0
python-bits: 64
OS: Linux
OS-release: 3.10.0-957.10.1.el7.x86_64
machine: x86_64
processor: x86_64
byteorder: little
LC_ALL: None
LANG: en_US.UTF-8
LOCALE: en_US.UTF-8

pandas: 0.23.4
pytest: 4.0.2
pip: 18.1
setuptools: 40.6.3
Cython: 0.29.2
numpy: 1.15.4
scipy: 1.1.0
pyarrow: None
xarray: None
IPython: 7.2.0
sphinx: 1.8.2
patsy: 0.5.1
dateutil: 2.7.5
pytz: 2018.7
blosc: None
bottleneck: 1.2.1
tables: 3.4.4
numexpr: 2.6.8
feather: None
matplotlib: 3.0.2
openpyxl: 2.5.12
xlrd: 1.2.0
xlwt: 1.3.0
xlsxwriter: 1.1.2
lxml: 4.2.5
bs4: 4.6.3
html5lib: 1.0.1
sqlalchemy: 1.2.15
pymysql: None
psycopg2: None
jinja2: 2.10
s3fs: None
fastparquet: None
pandas_gbq: None
pandas_datareader: None


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
filename = '/run/media/mnewlin/_userdata/uhnds/host/host_events_sample.txt'
df_host = pd.read_json(filename, orient='records', lines=True, dtype=str)
display(df_host)
#
df_host = df_host.replace(to_replace=[r"[\$]+"], value='', regex=True)
df_host = df_host.replace(to_replace=["nan"], value='none', regex=False)
df_host = df_host.replace(to_replace=["0x0"], value='success', regex=False)
df_host = df_host.drop(['Time'], axis=1)
display(df_host)
df_host_array = df_host.values

display(df_host.nunique(axis=0))

,AuthenticationPackage,Computer,DomainName,EventID,LogonID,LogonType,LogonTypeDescription,ServiceName,Source,Status,Time,UserName
0,nan,ActiveDirectory,Domain002,4769,nan,nan,nan,Comp883934$,Comp309534,0x0,2,User624729
1,MICROSOFT_AUTHENTICATION_PACKAGE_V1_0,ActiveDirectory,Domain002,4776,nan,nan,nan,nan,nan,0x0,2,Scanner
2,nan,ActiveDirectory,Domain002,4672,0x2e66398d,nan,nan,nan,nan,nan,2,ActiveDirectory$
3,Kerberos,ActiveDirectory,Domain002,4624,0x2e66398d,3.0,Network,nan,nan,nan,2,ActiveDirectory$
4,nan,ActiveDirectory,Domain002,4634,0x2e66398d,3.0,Network,nan,nan,nan,2,ActiveDirectory$
5,Kerberos,Comp966305,Domain002,4624,0x9f17415,3.0,Network,nan,nan,nan,2,User380010
6,nan,Comp966305,Domain002,4634,0x9f17415,3.0,Network,nan,nan,nan,2,User380010
7,Kerberos,Comp966305,Domain002,4624,0x9f17637,3.0,Network,nan,nan,nan,2,User096622
8,nan,Comp966305,Domain002,4634,0x9f17637,3.0,Network,nan,nan,nan,2,User096622
9,Kerberos,Comp966305,Domain002,4624,0x9f17fe4,3.0,Network,nan,nan,nan,2,User233472


,AuthenticationPackage,Computer,DomainName,EventID,LogonID,LogonType,LogonTypeDescription,ServiceName,Source,Status,UserName
0,none,ActiveDirectory,Domain002,4769,none,none,none,Comp883934,Comp309534,success,User624729
1,MICROSOFT_AUTHENTICATION_PACKAGE_V1_0,ActiveDirectory,Domain002,4776,none,none,none,none,none,success,Scanner
2,none,ActiveDirectory,Domain002,4672,0x2e66398d,none,none,none,none,none,ActiveDirectory
3,Kerberos,ActiveDirectory,Domain002,4624,0x2e66398d,3.0,Network,none,none,none,ActiveDirectory
4,none,ActiveDirectory,Domain002,4634,0x2e66398d,3.0,Network,none,none,none,ActiveDirectory
5,Kerberos,Comp966305,Domain002,4624,0x9f17415,3.0,Network,none,none,none,User380010
6,none,Comp966305,Domain002,4634,0x9f17415,3.0,Network,none,none,none,User380010
7,Kerberos,Comp966305,Domain002,4624,0x9f17637,3.0,Network,none,none,none,User096622
8,none,Comp966305,Domain002,4634,0x9f17637,3.0,Network,none,none,none,User096622
9,Kerberos,Comp966305,Domain002,4624,0x9f17fe4,3.0,Network,none,none,none,User233472


AuthenticationPackage    3
Computer                 2
DomainName               1
EventID                  5
LogonID                  5
LogonType                2
LogonTypeDescription     2
ServiceName              2
Source                   2
Status                   2
UserName                 6
dtype: int64

In [42]:
filename = '/run/media/mnewlin/_userdata/uhnds/host/extracted/wls_day-02.json'
num_samples = 1
data_df = None
test_filename = '/run/media/mnewlin/_userdata/uhnds/host/host_events_sample_2.txt'
reader = pd.read_json(filename, orient='records', lines=True, chunksize=1000)
for chunk in reader:
    display(chunk.head(20))
    print(chunk.shape)
    break

,AuthenticationPackage,Destination,DomainName,EventID,FailureReason,LogHost,LogonID,LogonType,LogonTypeDescription,ParentProcessID,...,ProcessID,ProcessName,ServiceName,Source,Status,SubjectDomainName,SubjectLogonID,SubjectUserName,Time,UserName
0,NaN,NaN,Domain001,4688,NaN,Comp883524,0x3e7,NaN,NaN,0x35c,...,0x1db4,wmiprvse.exe,NaN,NaN,NaN,NaN,NaN,NaN,86400,Comp883524$
1,NaN,NaN,Domain001,4688,NaN,Comp836249,0x45a5847,NaN,NaN,0x3a8c,...,0x3314,Proc542756.exe,NaN,NaN,NaN,NaN,NaN,NaN,86400,User805641
2,NaN,NaN,Domain001,4688,NaN,Comp397468,0x3e7,NaN,NaN,0x2b8,...,0xc770,svchost.exe,NaN,NaN,NaN,NaN,NaN,NaN,86400,Comp397468$
3,NaN,NaN,Domain001,4672,NaN,Comp916004,0x32bba0c8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86400,Comp916004$
4,Kerberos,NaN,Domain001,4624,NaN,Comp916004,0x32bba0c8,3.0,Network,NaN,...,NaN,NaN,NaN,Comp916004,NaN,NaN,NaN,NaN,86400,Comp916004$
5,NaN,NaN,Domain001,4688,NaN,Comp836249,0x3e7,NaN,NaN,0x1c0,...,0x17b8,conhost.exe,NaN,NaN,NaN,NaN,NaN,NaN,86400,Comp836249$
6,NaN,NaN,Domain001,4688,NaN,Comp000116,0x3e7,NaN,NaN,0x2e0,...,0x13ee0,rundll32.exe,NaN,NaN,NaN,NaN,NaN,NaN,86400,Comp000116$
7,NaN,NaN,Domain001,4634,NaN,Comp373973,0x13ff5c02,3.0,Network,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86400,Comp373973$
8,NaN,NaN,Domain001,4688,NaN,Comp763951,0x3e7,NaN,NaN,0x91c,...,0x190c,Proc461749.exe,NaN,NaN,NaN,NaN,NaN,NaN,86400,Comp763951$
9,NaN,NaN,nt authority,4688,NaN,EnterpriseAppServer,0x3e5,NaN,NaN,0x7d8,...,0x15c4,Proc965106.exe,NaN,NaN,NaN,NaN,NaN,NaN,86400,local service


(1000, 21)


In [179]:
filename = '/run/media/mnewlin/_userdata/uhnds/host/unconverted/real/samples_1000/wls_day-02.json_sample_0.txt'

json_data_all = pd.read_csv(filename, nrows=1000)

#display(json_data.head(10))
print(json_data_all.shape)
print(json_data_all.columns)
json_data_all = json_data_all.drop(['Time'], axis=1)
print(json_data_all.shape)
json_data_all.loc[:,'EventID'] = json_data_all.loc[:,'EventID'].astype(str)
json_data_all = json_data_all.replace(to_replace=[np.NaN], value='none', regex=False)
json_data_all = json_data_all.replace(to_replace=[r"[\$]+"], value='', regex=True)
json_data_all = json_data_all.replace(to_replace=["0x0"], value='success', regex=False)
#json_data_all = json_data_all.replace(to_replace=[r"^0x"], value='H', regex=True)
#json_data = json_data.replace(to_replace=[r"^[0-9]+"], value='ID', regex=True)
json_data = json_data_all.loc[:99]

(1000, 21)
Index(['AuthenticationPackage', 'Destination', 'DomainName', 'EventID',
       'FailureReason', 'LogHost', 'LogonID', 'LogonType',
       'LogonTypeDescription', 'ParentProcessID', 'ParentProcessName',
       'ProcessID', 'ProcessName', 'ServiceName', 'Source', 'Status',
       'SubjectDomainName', 'SubjectLogonID', 'SubjectUserName', 'Time',
       'UserName'],
      dtype='object')
(1000, 20)


In [180]:
display(json_data_all.loc[:,:'ParentProcessID'].head(20))
display(json_data_all.loc[:,'ParentProcessName':].head(20))

,AuthenticationPackage,Destination,DomainName,EventID,FailureReason,LogHost,LogonID,LogonType,LogonTypeDescription,ParentProcessID
0,none,none,Domain001,4688,none,Comp883524,0x3e7,none,none,0x35c
1,none,none,Domain001,4688,none,Comp836249,0x45a5847,none,none,0x3a8c
2,none,none,Domain001,4688,none,Comp397468,0x3e7,none,none,0x2b8
3,none,none,Domain001,4672,none,Comp916004,0x32bba0c8,none,none,none
4,Kerberos,none,Domain001,4624,none,Comp916004,0x32bba0c8,3,Network,none
5,none,none,Domain001,4688,none,Comp836249,0x3e7,none,none,0x1c0
6,none,none,Domain001,4688,none,Comp000116,0x3e7,none,none,0x2e0
7,none,none,Domain001,4634,none,Comp373973,0x13ff5c02,3,Network,none
8,none,none,Domain001,4688,none,Comp763951,0x3e7,none,none,0x91c
9,none,none,nt authority,4688,none,EnterpriseAppServer,0x3e5,none,none,0x7d8


,ParentProcessName,ProcessID,ProcessName,ServiceName,Source,Status,SubjectDomainName,SubjectLogonID,SubjectUserName,UserName
0,svchost,0x1db4,wmiprvse.exe,none,none,none,none,none,none,Comp883524
1,Proc028501,0x3314,Proc542756.exe,none,none,none,none,none,none,User805641
2,services,0xc770,svchost.exe,none,none,none,none,none,none,Comp397468
3,none,none,none,none,none,none,none,none,none,Comp916004
4,none,none,none,none,Comp916004,none,none,none,none,Comp916004
5,csrss,0x17b8,conhost.exe,none,none,none,none,none,none,Comp836249
6,services,0x13ee0,rundll32.exe,none,none,none,none,none,none,Comp000116
7,none,none,none,none,none,none,none,none,none,Comp373973
8,Proc461749,0x190c,Proc461749.exe,none,none,none,none,none,none,Comp763951
9,Proc126308,0x15c4,Proc965106.exe,none,none,none,none,none,none,local service


In [183]:
def calculate_sample_tf(df):
    column_term_list = []
    for col in df.columns:
        col_terms = Counter(df.loc[:,col])
        column_term_list.append(col_terms)
    return column_term_list
    
def calculate_tf(counter):
    num_words = sum(counter.values())
    tf_dict = {}
    for item in counter:
        tf_dict[item] = counter[item]/float(num_words)
    return tf_dict

def calculate_idf(docList):
    N = len(docList)
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            word = str(word)
            if val > 0:
                if word in idfDict.keys():
                    idfDict[word] += 1
                else:
                    idfDict[word] = 1
    for word, val in idfDict.items():
        word = str(word)
        idfDict[word] = np.log10(N/float(val))
    return idfDict

def compute_tfidf(tf_dict, idf_dict):
    tfidf_dict = {}
    for word, val in tf_dict.items():
        tfidf_dict[word] = val * idf_dict[word]
    return tfidf_dict

def mergeDict(dict1, dict2):
    ''' Merge dictionaries and add values of common keys in list'''
    dict3 = {**dict1, **dict2}
    for key, value in dict3.items():
        if key in dict1 and key in dict2:
               dict3[key] = value + dict1[key]
 
    return dict3

In [185]:
#term_list = ['term1', 'term2', 'term3']
#counts = [1,2,3]
#tf_dict = dict(zip(term_list, counts))
#print(tf_dict)

doc_1 = json_data_all.loc[:99].values.flatten().astype(str)
doc_2 = json_data_all.loc[100:199].values.flatten().astype(str)
doc_3 = json_data_all.loc[200:299].values.flatten().astype(str)

all_terms_1 = {}
all_terms_1 = Counter(doc_1)
#print(sum(all_terms_1.values()))
tf_dict = calculate_tf(all_terms_1)
#print(tf_dict)

all_terms_2 = Counter(doc_2)
tf_dict_2 = calculate_tf(all_terms_2)
#print(tf_dict_2)
all_terms_3 = Counter(doc_3)
tf_dict_3 = calculate_tf(all_terms_3)
#print(tf_dict_2)

#print(dict(all_terms_1))
docList = [dict(all_terms_1), dict(all_terms_2), dict(all_terms_3)]
#print(docList[0])
#print(docList[1])
idf_all = calculate_idf(docList)
#print(idf_all)

tfidf_1 = compute_tfidf(dict(all_terms_1), idf_all)
print(tfidf_1)


{'none': 0.0, 'Domain001': 0.0, '4688': 0.0, 'Comp883524': 0.9542425094393249, '0x3e7': 0.0, '0x35c': 0.47712125471966244, 'svchost': 0.0, '0x1db4': 0.47712125471966244, 'wmiprvse.exe': 0.5282737771670437, 'Comp836249': 1.2326388133897686, '0x45a5847': 0.47712125471966244, '0x3a8c': 0.47712125471966244, 'Proc028501': 0.47712125471966244, '0x3314': 0.47712125471966244, 'Proc542756.exe': 0.47712125471966244, 'User805641': 0.47712125471966244, 'Comp397468': 0.9542425094393249, '0x2b8': 0.17609125905568124, 'services': 0.0, '0xc770': 0.47712125471966244, 'svchost.exe': 0.0, '4672': 0.0, 'Comp916004': 2.465277626779537, '0x32bba0c8': 0.9542425094393249, 'Kerberos': 3.8169700377572995, '4624': 0.0, '3.0': 0.0, 'Network': 0.0, '0x1c0': 0.47712125471966244, 'csrss': 0.0, '0x17b8': 0.47712125471966244, 'conhost.exe': 0.0, 'Comp000116': 0.9542425094393249, '0x2e0': 0.0, '0x13ee0': 0.47712125471966244, 'rundll32.exe': 0.0, '4634': 1.9370038496124935, 'Comp373973': 0.9542425094393249, '0x13ff5c02'

'\nterms = Counter(json_data.iloc[:,0])\nprint(terms)\nfor item in terms:\n    print("{}:{}".format(item,terms[item]))\nprint(len(list(terms.elements())))\n'

In [187]:
doc = json_data_all.loc[:99]
docList = []
doc_tfs = []
for col in doc.columns:
    d = doc.loc[:,col].astype(str)
    tf_col = calculate_tf(Counter(d))
    doc_tfs.append(tf_col)
    docList.append(dict(tf_col))
idf_doc = calculate_idf(docList)
doc_tfidfs = []
for tf in doc_tfs:
    doc_tfidf = compute_tfidf(tf, idf_doc)
    doc_tfidfs.append(doc_tfidf)


In [188]:
print(doc_tfidfs[0])

{'none': 0.06589880884547837, 'Kerberos': 0.1040823996531185, 'Negotiate': 0.11709269960975831, 'MICROSOFT_AUTHENTICATION_PACKAGE_V1_0': 0.1040823996531185, 'NTLM': 0.0910720996964787}


In [194]:
tfidf_vals = np.zeros(doc.values.shape)
for j in range(tfidf_vals.shape[1]):
    curr_col = doc.iloc[:,j].astype(str)
    for i in range(tfidf_vals.shape[0]):
        col_tfidf = doc_tfidfs[j]
        tfidf_vals[i,j] = col_tfidf[curr_col[i]]
converted_data = pd.DataFrame(data=tfidf_vals, columns=doc.columns)
display(converted_data.head())
print(converted_data.columns)

,AuthenticationPackage,Destination,DomainName,EventID,FailureReason,LogHost,LogonID,LogonType,LogonTypeDescription,ParentProcessID,ParentProcessName,ProcessID,ProcessName,ServiceName,Source,Status,SubjectDomainName,SubjectLogonID,SubjectUserName,UserName
0,0.065899,0.095941,0.5,0.416330,0.09691,0.010000,0.460000,0.062992,0.062992,0.013010,0.052041,0.013010,0.039031,0.09691,0.073652,0.089157,0.095941,0.095941,0.095941,0.010000
1,0.065899,0.095941,0.5,0.416330,0.09691,0.050000,0.013010,0.062992,0.062992,0.013010,0.013010,0.013010,0.013010,0.09691,0.073652,0.089157,0.095941,0.095941,0.095941,0.013010
2,0.065899,0.095941,0.5,0.416330,0.09691,0.010000,0.460000,0.062992,0.062992,0.013010,0.247196,0.013010,0.052041,0.09691,0.073652,0.089157,0.095941,0.095941,0.095941,0.010000
3,0.065899,0.095941,0.5,0.312247,0.09691,0.048928,0.026021,0.062992,0.062992,0.065899,0.065899,0.056208,0.056208,0.09691,0.073652,0.089157,0.095941,0.095941,0.095941,0.027959
4,0.104082,0.095941,0.5,0.312247,0.09691,0.048928,0.026021,0.338268,0.338268,0.065899,0.065899,0.056208,0.056208,0.09691,0.013979,0.089157,0.095941,0.095941,0.095941,0.027959


Index(['AuthenticationPackage', 'Destination', 'DomainName', 'EventID',
       'FailureReason', 'LogHost', 'LogonID', 'LogonType',
       'LogonTypeDescription', 'ParentProcessID', 'ParentProcessName',
       'ProcessID', 'ProcessName', 'ServiceName', 'Source', 'Status',
       'SubjectDomainName', 'SubjectLogonID', 'SubjectUserName', 'UserName'],
      dtype='object')


In [92]:

# Create first dictionary
dict1 = {  'Ritika': 5, 'Sam': 7, 'John' : 10 }
 
# Create second dictionary
dict2 = {'Aadi': 8,'Sam': 20,'Mark' : 11 }

def mergeDict(dict1, dict2):
    ''' Merge dictionaries and keep values of common keys in list'''
    dict3 = {**dict1, **dict2}
    for key, value in dict3.items():
        if key in dict1 and key in dict2:
               dict3[key] = value + dict1[key]
 
    return dict3
 
# Merge dictionaries and add values of common keys in a list
dict3 = mergeDict(dict1, dict2)
 
print('Dictionary 3 :')
print(dict3)

Dictionary 3 :
{'Ritika': 5, 'Sam': 27, 'John': 10, 'Aadi': 8, 'Mark': 11}
